In [134]:
# Imports
import pandas as pd
import numpy as np
import optuna
import re
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [135]:
# Function to normalize the format for the datasets
def normalizeDatasetFormat(dataset):
    # 14 most lucrative/popular videogame franchises
    topFranchises = [
        "Mario", "Tetris", "Call of Duty", "Pokemon", "Grand Theft Auto",
        "FIFA", "Wii", "Lego", "The Sims", "Assassin's Creed", "Final Fantasy",
        "Sonic", "Zelda", "Resident Evil"
    ]

    # Merge all the franchises in one same regex expresion
    regex = '|'.join([re.escape(franchise) for franchise in topFranchises])

    # Apply the regex to extract the franchise on each title name
    dataset['Franchise'] = (
        dataset['Name']
        .str.extract(f'({regex})', flags=re.IGNORECASE, expand=False)
        .fillna('Other')
    )

    # Normalize franchise name, first upper case rest lower case
    dataset['Franchise'] = dataset['Franchise'].str.title()

    # Get only the 14 most common platforms
    topPlatforms = list(dataset['Platform'].value_counts().nlargest(14).index)
    print(topPlatforms)

    # Get the 14 most common platforms and for all others assign 'Other' ending with a 15 cardinality.
    dataset['PlatformReduced'] = dataset['Platform'].apply(lambda x: x if x in topPlatforms else 'Other')
    dataset['PlatformReduced'].value_counts()

    # Get only the 14 most common publishers
    topPublishers = list(dataset['Publisher'].value_counts().nlargest(14).index)
    print(topPublishers)

    # Get the 14 most common publishers and for all others assign 'Other' ending with a 15 cardinality.
    dataset['PublisherReduced'] = dataset['Publisher'].apply(lambda x: x if x in topPublishers else 'Other')
    dataset['PublisherReduced'].value_counts()

    return dataset

In [136]:
# Function to acquire the best parameters for Random Forest Regressor
def bestParamsEstimatorRanFor(preprocessor, X_train, y_train):
    # Creation of staple model to work with
    model = RandomForestRegressor(random_state=0, n_jobs = -1)

    # Pipeline creation to use in GridSearchCV
    regressor = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Parameters that are going to be tested during GridSearchCV
    param_grid = {
        'model__n_estimators': [90, 95, 100, 105, 110],
        'model__max_depth': [15, 20, 25]
    }
        
    # Declaration of grid search with parameters, cv = 5 cross validation in 5 splits
    grid_search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')

    # Search for the best parameters during the fit of the grid search with the train dataset.
    grid_search.fit(X_train, y_train)

    print("Best Parameters:", grid_search.best_params_)

In [165]:
# Function to acquire the best parameters for XGB Regressor (Wrap function to accept parameters)
def bestParamsEstimatorXGBR(preprocessor, X_train, y_train):
    # Necessary function for optuna
    def objective(trial):
        # Declaration parameters and ranges for each one that are going to be tested with optuna
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 185, 190),
            'max_depth': trial.suggest_int('max_depth', 3, 5),
            'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.2),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'random_state': 0
        }

        # Declaration of XGBR model to be used with optuna
        model = XGBRegressor(**params)

        # Pipeline creation to use with optuna
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])

        # Cross validation scores acquired and mean calculated, needed for optuna to find best parameters.
        score = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
        return score.mean()
    
    return objective

In [138]:
# Getting the train dataset 
X = pd.read_csv('D:/Archivos Personales/Courses/Data Science/Projects and Competitions/Video Game Sales/vgsales.csv')

# Remove any rows that may have NaN values in NA_Sales column.
X.dropna(axis=0, subset=['NA_Sales'], inplace=True)

# Assigning the target column to Y from X
Y = X.NA_Sales
X.drop(['NA_Sales'], axis=1, inplace=True)

# Splitting the dataset into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [139]:
# Review how many NaN values are per columns, to see if dropping a column can be an option.
nanValuesPerColumn = X.isnull().sum()
nanValuesPerColumn

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [140]:
# Get the categorical values in the dataset and see their cardinality
categorical_columns = list(X_train_full.select_dtypes(include=['object']).columns)
X_train_full[categorical_columns].nunique()

Name         9677
Platform       30
Genre          12
Publisher     522
dtype: int64

In [141]:
# Analyze the quantity for each platform
X_train_full['Platform'].value_counts()

Platform
PS2     1758
DS      1730
Wii     1060
PS3     1053
X360    1026
PSP      962
PS       952
PC       775
XB       663
GBA      649
GC       448
3DS      410
PSV      320
PS4      264
N64      259
SNES     185
XOne     167
SAT      137
WiiU     113
2600     107
NES       80
GB        76
DC        40
GEN       22
NG        10
SCD        4
WS         4
3DO        2
PCFX       1
GG         1
Name: count, dtype: int64

In [142]:
# Normalize the transformation and data format in all datasets
X_train_full = normalizeDatasetFormat(X_train_full)
X_valid_full = normalizeDatasetFormat(X_valid_full)
X = normalizeDatasetFormat(X)

['PS2', 'DS', 'Wii', 'PS3', 'X360', 'PSP', 'PS', 'PC', 'XB', 'GBA', 'GC', '3DS', 'PSV', 'PS4']
['Electronic Arts', 'Activision', 'Namco Bandai Games', 'Ubisoft', 'Konami Digital Entertainment', 'THQ', 'Nintendo', 'Sony Computer Entertainment', 'Sega', 'Take-Two Interactive', 'Capcom', 'Atari', 'Tecmo Koei', 'Warner Bros. Interactive Entertainment']
['DS', 'PS2', 'PS3', 'Wii', 'PSP', 'PS', 'X360', 'PC', 'GBA', 'XB', 'GC', '3DS', 'PSV', 'PS4']
['Electronic Arts', 'Activision', 'Ubisoft', 'Namco Bandai Games', 'Konami Digital Entertainment', 'THQ', 'Nintendo', 'Sony Computer Entertainment', 'Sega', 'Atari', 'Take-Two Interactive', 'Capcom', 'Tecmo Koei', 'Square Enix']
['DS', 'PS2', 'PS3', 'Wii', 'X360', 'PSP', 'PS', 'PC', 'XB', 'GBA', 'GC', '3DS', 'PSV', 'PS4']
['Electronic Arts', 'Activision', 'Namco Bandai Games', 'Ubisoft', 'Konami Digital Entertainment', 'THQ', 'Nintendo', 'Sony Computer Entertainment', 'Sega', 'Take-Two Interactive', 'Capcom', 'Atari', 'Tecmo Koei', 'Square Enix']


In [143]:
# Get the categorical values once again but with thew new columns and see their cardinality
categorical_columns = list(X_train_full.select_dtypes(include=['object']).columns)
X_train_full[categorical_columns].nunique()

Name                9677
Platform              30
Genre                 12
Publisher            522
Franchise             15
PlatformReduced       15
PublisherReduced      15
dtype: int64

In [144]:
# Variable declaration for final categorical and numerical columns
final_categorical_columns = []
final_numerical_columns = []

# Extract only categorical columns and numerical columns and append them to their respective variables
for column in X_train_full.columns:
    if X_train_full[column].nunique() < 16 and X_train_full[column].dtype == 'object':
        final_categorical_columns.append(column)
    elif X_train_full[column].dtype in ['int64', 'float64']:
        final_numerical_columns.append(column)

print(f"Final categorical columns: {final_categorical_columns}\nFinal numerical columns: {final_numerical_columns}")

Final categorical columns: ['Genre', 'Franchise', 'PlatformReduced', 'PublisherReduced']
Final numerical columns: ['Rank', 'Year', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']


In [145]:
# Final total columns to keep in final version of the datasets
myColumns = final_categorical_columns + final_numerical_columns

# Normalization of columns for all datasets
X_train = X_train_full[myColumns].copy()
X_valid = X_valid_full[myColumns].copy()
X = X[myColumns].copy()

X_train.head()

,Genre,Franchise,PlatformReduced,PublisherReduced,Rank,Year,EU_Sales,JP_Sales,Other_Sales,Global_Sales
11199,Action,Lego,GBA,Other,11201,2006.0,0.02,0.00,0.0,0.09
11648,Misc,Other,PS2,Namco Bandai Games,11650,2009.0,0.00,0.08,0.0,0.08
14120,Shooter,Other,Other,Other,14122,1998.0,0.00,0.03,0.0,0.03
14933,Strategy,Other,PC,Other,14936,1998.0,0.02,0.00,0.0,0.02
12119,Sports,Other,GC,Sega,12121,2002.0,0.01,0.00,0.0,0.07


In [146]:
#------------------------------------------------Pipeline creation----------------------------------------------------

# Creation of the numerical transformer
numericalTransformer = SimpleImputer(strategy='most_frequent')

# Creation of the categorical transformer
categoricalTransformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('OneHotEnc', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Creation of the preprocessor to use in the pipeline as well as in the best parameters functions
preprocessor = ColumnTransformer(transformers=[
    ('numerical', numericalTransformer, final_numerical_columns),
    ('categorical', categoricalTransformer, final_categorical_columns)
])

#bestParamsEstimatorRanFor(preprocessor, X_train, y_train)

In [147]:
# Best parameters using bestParamsEstimatorRanFor function: {'model__max_depth': 25, 'model__n_estimators': 100}
model = RandomForestRegressor(n_estimators=100, max_depth=25, random_state=0) 

# Pipeline creation with model with best parameters
regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Fitting and training model with train dataset
regressor.fit(X_train, y_train)

# Get predictions for valid dataset with random forest regressor model
predictions = regressor.predict(X_valid)

In [148]:
# Calculated error metrics for R2, RMSE and MAE
rmseRF = np.sqrt(mean_squared_error(y_valid, predictions))
maeRF = mean_absolute_error(y_valid, predictions)
r2RF = r2_score(y_valid, predictions)

print(f"R2: {r2RF*100:.2f}%") # Closer to 100% better.
print(f"RMSE: {rmseRF}") # Closer to 0 better. Useful if big overestimations/costs are a big issue for example. (Be careful with outliers)
print(f"MAE: {maeRF}") # Closer to 0 better. More generalized, doesn't consider big underestimations or overestimations.

R2: 89.12%
RMSE: 0.2797627174264075
MAE: 0.02612368130923189


In [168]:
# Study creation and optimization process to find best parameters with optuna
study = optuna.create_study(direction='maximize')
study.optimize(bestParamsEstimatorXGBR(preprocessor, X_train, y_train), n_trials=100)

print("Best parameters found:", study.best_params)

[I 2025-04-24 01:45:30,234] A new study created in memory with name: no-name-7e991fb7-00fe-42e3-8573-4aa7fc0eb4bb
[I 2025-04-24 01:45:36,392] Trial 0 finished with value: -0.1267659515196566 and parameters: {'n_estimators': 187, 'max_depth': 4, 'learning_rate': 0.1699381481911377, 'subsample': 0.9594407245226749, 'colsample_bytree': 0.9280491669635962}. Best is trial 0 with value: -0.1267659515196566.
[I 2025-04-24 01:45:42,685] Trial 1 finished with value: -0.12556062305225577 and parameters: {'n_estimators': 190, 'max_depth': 4, 'learning_rate': 0.172294832971713, 'subsample': 0.8449361960226623, 'colsample_bytree': 0.565334405501966}. Best is trial 1 with value: -0.12556062305225577.
[I 2025-04-24 01:45:48,495] Trial 2 finished with value: -0.1281094142730414 and parameters: {'n_estimators': 186, 'max_depth': 3, 'learning_rate': 0.17091524704466807, 'subsample': 0.8955878548309084, 'colsample_bytree': 0.7038479005299306}. Best is trial 1 with value: -0.12556062305225577.
[I 2025-04-

Best parameters found: {'n_estimators': 188, 'max_depth': 4, 'learning_rate': 0.169647227120005, 'subsample': 0.5606818367361254, 'colsample_bytree': 0.9031277598760848}


In [170]:
# Creation of XGBRegressor with the best parameters found by optuna
model = XGBRegressor(
    n_estimators = study.best_params['n_estimators'],
    max_depth = study.best_params['max_depth'],
    learning_rate = study.best_params['learning_rate'],
    subsample = study.best_params['subsample'],
    colsample_bytree = study.best_params['colsample_bytree']
    )

# Pipeline creation with model with best parameters
regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Fitting and training model with train dataset
regressor.fit(X_train, y_train)

# Get predictions for valid dataset with random forest regressor model
predictions = regressor.predict(X_valid)

In [171]:
# Calculated error metrics for R2, RMSE and MAE
rmseRF = np.sqrt(mean_squared_error(y_valid, predictions))
maeRF = mean_absolute_error(y_valid, predictions)
r2RF = r2_score(y_valid, predictions)

print(f"R2: {r2RF*100:.2f}%") # Closer to 100% better.
print(f"RMSE: {rmseRF}") # Closer to 0 better. Useful if big overestimations/costs are a big issue for example. (Be careful with outliers)
print(f"MAE: {maeRF}") # Closer to 0 better. More generalized, doesn't consider big underestimations or overestimations.

R2: 84.04%
RMSE: 0.33878002398320384
MAE: 0.03603258270885044
